# Import potrzebnych biblotek 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyedflib 
import pandas as pd
import mne


# Ułożenie elektrod na głowie 

In [37]:
#show the 64 channel montage
img = plt.imread('files/64_channel_sharbrough-old.png')
plt.imshow(img)
plt.axis('off')
plt.show()


# conversion data type form .edf to data frame

In [38]:
subject = 1
file = 5
fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"


def file_to_DataDrame(path):
    """
    This function takes in a file path and returns a dataframe with the data and the target values
    format:
        Fc5	        Fc3	        Fc1	        ...	Oz	        O2	        Iz	        target
    0	-0.000046	-0.000041	-0.000032	...	0.000040	0.000108	0.000055	T0
    1	-0.000054	-0.000048	-0.000034	...	0.000064	0.000114	0.000074	T0
    ...
    """

    reader = mne.io.read_raw_edf(path, preload=True)
    annotations = reader.annotations  # get the values of the annotations
    codes = annotations.description  # get the codes from the annotations

    df = pd.DataFrame(
        reader.get_data().T,
        columns=[channel.replace(".", "") for channel in reader.ch_names],
    )  # transpose the data to get the right shape
    df = df[~(df == 0).all(axis=1)]  # remove rows with all zeros
    timeArray = np.array(
        [round(x, 10) for x in np.arange(0, len(df) / 160, 0.00625)]
    )  # create an array of time values

    codeArray = []
    counter = 0
    for timeVal in timeArray:
        if (
            timeVal in annotations.onset
        ):  # if the time value is in the onset array, add the corresponding code to the codeArray
            counter += 1
        code_of_target = int(
            codes[counter - 1].replace("T", "")
        )  # convert T0 to 0, T1 to 1, etc
        codeArray.append(code_of_target)

    df["target"] = np.array(codeArray).T
    return df

In [39]:
df = file_to_DataDrame(fileName)

In [40]:
df.head(2)


In [41]:
df.tail(2)

In [42]:
# check presiction data type
print(f"data presiction in df:\t {df.iloc[0,0].dtype}")
a = df.iloc[1].values  # get the first row of the dataframe
a = a[:-1]  # remove the target value
a = a.astype(np.float64)  # convert the values to float
print(f"class type of a:\t {type(np.array(a))}")
print(f"schape of a:\t\t {a.shape}")
print(f"data presiction in a:\t {a.dtype}")
print(a)

# Create a new dataframe with the selected ekperiment numbers

In [43]:
#create a new dataframe with num of ecperiments 3,4,7,8,11,12
choose_num_of_exp = [3,4,7,8,11,12]
# person = renage (1,109)
person = 1

#conect all experiments to one dataframe np.array 6x20000x64x1 
all_meseurments = np.zeros((len(choose_num_of_exp),20000,65))
for i in range(6):
    fileName = f"files/S{person:03d}/S{person:03d}R{choose_num_of_exp[i]:02d}.edf"
    df = file_to_DataDrame(fileName)
    for j in range(20000):
        if j >= len(df):
            all_meseurments[i][j] = np.zeros(65) 
        else:
            all_meseurments[i][j] = df.iloc[j].values
      
          

In [ ]:
#create a new dataframe with num of ecperiments 3,4,7,8,11,12
choose_num_of_exp = [3,4,7,8,11,12]
# person = renage (1,109)
person = 1
array_measurement = np.zeros((64,1))
all_measurements = np.zeros((len(choose_num_of_exp),64,1))
for i in choose_num_of_exp:
    fileName = f'files/S{person:03d}/S{person:03d}R{i:02d}.edf'
    df = file_to_DataDrame(fileName)
    array_measurement = np.array(df.iloc[i].values[:-1], dtype=np.float64) #get the first row of the dataframe and remove the target value and convert the values to np.array
    array_target = np.array(df.iloc[i].values[-1])  #get the target value
    array_measurement_target = np.append(array_measurement, [array_target], axis=0)
    print(array_measurement_target.shape)

person = 1
all_measurements = []

for i in choose_num_of_exp:
    fileName = f'files/S{person:03d}/S{person:03d}R{i:02d}.edf'
    df = file_to_DataDrame(fileName)
    array_measurement = np.array(df.iloc[i].values[:-1], dtype=np.float64)
    array_target = np.array(df.iloc[i].values[-1])
    array_measurement_target = np.append(array_measurement, [array_target], axis=0)
    print(array_measurement_target.shape)
    all_measurements.append(array_measurement_target)

# Convert list of numpy arrays back to numpy array
# all_measurements = np.array(all_measurements), [array_measurement_target], axis=0)
    
    


In [44]:
print(df['Fc5'][1])
print(df.head(2))

In [45]:
print(df.tail(2))

In [46]:
# Start T0 or T1 or T2 
def index_of_TX(df):
    key = 0
    dict_TX_index = {key: [df['target'][0], 0] }
    current_TX = df['target'][0]
    for i in range(len(df['target'])):
        if df['target'][i] != current_TX:
            key += 1
            dict_TX_index[key] = [df['target'][i], i]
            current_TX = df['target'][i]
 
    return dict_TX_index
   
        
print(index_of_TX(df))

In [ ]:
indexs = index_of_TX(df)
xT0=[]
xT1=[]
xT2=[]
ax= df.plot(y='Fc5')
ax.set_xlabel("Time [s]")
for i in range(0,len(indexs)):
    if indexs[i][0] == 0:
        xT0.append(indexs[i][1])      
    elif indexs[i][0] == 1:
        xT1.append(indexs[i][1])
    elif indexs[i][0] == 2:
        xT2.append(indexs[i][1])
#doter line
ax.vlines(x=xT0, ymin=-0.0003, ymax=0.0003, color='r',label='T0',linestyles='dashed')
ax.vlines(x=xT1, ymin=-0.0003, ymax=0.0003, color='b',label='T1',linestyles='dashed')
ax.vlines(x=xT2, ymin=-0.0003, ymax=0.0003, color='y',label='T2', linestyles='dashed')
ax.legend()



# Normalization df

In [47]:
#
def normalize_data(df):
    df = (df - df.min()) / (df.max() - df.min())
    return df

In [48]:
df = normalize_data(df)

In [49]:
indexs = index_of_TX(df)
xT0=[]
xT1=[]
xT2=[]
ax= df.plot(y='Fc5')
ax.set_xlabel("Time [s]")
for i in range(0,len(indexs)):
    if indexs[i][0] == 0:
        xT0.append(indexs[i][1])      
    elif indexs[i][0] == 1:
        xT1.append(indexs[i][1])
    elif indexs[i][0] == 2:
        xT2.append(indexs[i][1])
#doter line
#max value
max_value = df['Fc5'].max()
ax.vlines(x=xT0, ymin=-0, ymax=max_value, color='r',label='T0',linestyles='dashed')
ax.vlines(x=xT1, ymin=-0, ymax=max_value, color='b',label='T1',linestyles='dashed')
ax.vlines(x=xT2, ymin=-0, ymax=max_value, color='y',label='T2', linestyles='dashed')
ax.legend()



In [50]:
# Najpierw zainstaluj pyEDFlib, jeśli jeszcze tego nie zrobiłeś:
# !pip install pyedflib



# Zastąp 'path_to_edf_file.edf' ścieżką do Twojego pliku EDF
file_path = '/home/daniel/repos/Decoding_of_EEG/files/S001/S001R05.edf'

# Odczytanie pliku EDF
with pyedflib.EdfReader(file_path) as f:
    # Pobranie liczby sygnałów
    n = f.signals_in_file

    # Pobranie etykiet sygnałów
    signal_labels = f.getSignalLabels()

    signals = []
    # Odczytanie i wyplotowanie każdego sygnału
    for i in range(n):
        signal = f.readSignal(i)
        signals.append(signal)
        plt.figure(figsize=(12, 4))
        plt.plot(signal)
        plt.title(signal_labels[i])
        plt.show()
        if i < 1:
            break
    
    signals_array = np.array(signals)
    



In [ ]:
print(signal_labels)
signal= f.readSignal


In [51]:
print(signals)

In [ ]:
print(np.max(signals_array))
print(np.min(signals_array))
